In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import csv
from random import randrange
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import float32
import warnings
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')

In [ ]:
def distanceNorm(Norm,D_value):  
    # Norm for distance  
    if Norm == '1':  
        counter = np.absolute(D_value);    
        counter = np.sum(counter);  
    elif Norm == '2':  
        counter = np.power(D_value,2);  
        counter = np.sum(counter);  
        counter = np.sqrt(counter);  
    else:  
        raise Exception('We will program this later......');  
    return counter;  

In [ ]:
from numpy import float32
file = '初筛后的基因数据.csv'
expression = np.loadtxt(file, dtype=float32, delimiter=",")
features = np.array(expression[:,:-1])
label = np.array(expression[:,-1], dtype=int)
labels = np.zeros(len(label),dtype=int)
for i in range(len(labels)):
    labels[i]=label[i]-1

In [ ]:
import random
a=list()
count = [0,0,0,0,0,0]
(n_samples,n_features) = np.shape(features)
for index in range(n_samples):
    a.append([features[index],index,labels[index]])
a.sort(key = lambda x:x[2])
for i in a:
    count[i[2]] += 1
left = 0
right =count[0]-1
location = list()
for i in range(len(set(labels))):
    location += [random.randint(left,right) for _ in range(5)]    
    left = left+count[i]
    if i<5:
        right = right+count[i+1]
    else:
        right = 0                                                   

In [ ]:
def fit(features,labels,k,norm):     
    (n_samples,n_features) = np.shape(features)
    distance = np.zeros((n_samples,n_samples))
    weight = np.zeros(n_features) 
    for i in range(n_samples):
        for j in range(n_samples):
            D_value = features[i] - features[j]
            if i == j:
                distance[i,j] = 10000      
            else:
                distance[i,j] = distanceNorm(norm,D_value)
    
    for i in location:
        one_sample = a[i][1] 
        one_feature = features[one_sample]
        pr = count[a[i][2]]/n_samples
        distance_sort =list()
        nearHit = list() 
        nearMiss = dict()   
        n_labels = list(set(labels))
        termination = np.zeros(len(n_labels),dtype = int) 
        del n_labels[n_labels.index(labels[one_sample])]  
        for i in n_labels:
            nearMiss[i] = list()  
        
        for index in range(n_samples):
            distance_sort.append([distance[one_sample,index],index,labels[index]])
        distance_sort.sort(key = lambda x:x[0]) 
        
        for index in range (n_samples):
            if distance_sort[index][2]==labels[one_sample]:
                if len(nearHit) < k:
                    nearHit.append(features[distance_sort[index][1]])
                else:
                    termination[distance_sort[index][2]] =1   
            elif distance_sort[index][2] != labels[one_sample]:
                if len(nearMiss[distance_sort[index][2]]) < k:
                    nearMiss[distance_sort[index][2]].append(features[distance_sort[index][1]])
                else:
                    termination[distance_sort[index][2]] =1
            if list(termination).count(0) == 0:      
                break
        
        nearHit_term = np.zeros(n_features)
        for i in nearHit:
            nearHit_term +=np.abs(np.power(one_feature-i,2))
        nearMiss_term =np.zeros((len(n_labels),n_features))
        for index,label in enumerate(nearMiss.keys()):  
            for x in nearMiss[label]:
                nearMiss_term[index] +=  np.abs(np.power(one_feature-x,2))
            pc = count[label]/n_samples
            weight += (pc/(1-pr))*nearMiss_term[index]/(k*len(nearMiss.keys()))
        weight -= nearHit_term/k
    
    return weight/(len(location))

In [ ]:
acc_list =list()

def main():
   
    n_samples, n_features = features.shape    
    kf =KFold(n_splits=10)
    feature_number_list =list(range(5,21,1))+list(range(30,101,10))
    
    for feature_number in feature_number_list:
        clf = RandomForestClassifier(n_estimators=200,max_features='sqrt', oob_score=True,random_state=120)
        correct = 0
        for train, test in kf.split(features):
            score = fit(features, labels,k=5,norm='2') 
            idx = feature_ranking(score)  
            selected_features = features[:, idx[0:feature_number]] 
            clf.fit(selected_features[train], labels[train])  
            y_predict = clf.predict(selected_features[test])
            acc = accuracy_score(labels[test], y_predict)
            correct = correct + acc
        acc_list.append(float(correct)/10)
        print('Accuracy:{}'.format(float(correct)/10))
    return acc_list
main()